## UFRJ | Cálculo Numérico | 2021.2

### Tarefa 6 | Gabriele Jandres | 119159948

#### Grupo (Q1 e Q3): Francisco Oliveira, Gabriele Jandres e Victor Cardoso

#### Discussões com: Carlos Bravo, Francisco Oliveira, Victor Cardoso

In [1]:
using LinearAlgebra
using Polynomials: Polynomial

### Exercício 1 | Biblioteca

_________________________________________________________________________________________________________________________________________________________________________________________

#### Funções da biblioteca

##### i. Diagonal

In [2]:
# Função para resolver um sistema diagonal

# -- Parâmetros --
# A: matriz diagonal n x n
# b: matriz coluna de resultados

# -- Retorno --
# x: x tal que Ax = b

function resolve_diagonal(A, b) 
    n, = size(A) # dimensão da matriz A
    x = zeros(n, 1) # criamos uma matriz coluna n x 1
    
    for i = 1:n
        x[i] = b[i] / A[i, i] # elemento da matriz de resultados dividido pela posição correspondente na diagonal
    end
    
    return x
end

resolve_diagonal (generic function with 1 method)

##### ii. Triangular superior

In [3]:
# Função para resolver um sistema triangular superior n x n por susbtituição reversa

# -- Parâmetros --
# U: matriz triangular onde os elementos da diagonal são diferentes de zero
# b: matriz coluna de resultados

# -- Retorno --
# x: x tal que Ux = b

function resolve_triangular_superior(U, b)     
    n, = size(U) # dimensão da matriz U
    x = zeros(n, 1) # criamos uma matriz coluna n x 1
    
    # resolução do sistema por substituição reversa
    for i = reverse(1:n)
        x[i] = b[i]
        for j = reverse(i + 1:n)
            x[i] -= U[i, j] * x[j]
        end
        x[i] /= U[i, i]
    end
    
    return x 
end

resolve_triangular_superior (generic function with 1 method)

##### iii. Triangular inferior

In [4]:
# Função para resolver um sistema triangular inferior n x n por substituição direta

# -- Parâmetros --
# L: matriz triangular onde os elementos da diagonal são diferentes de zero
# b: matriz coluna de resultados

# -- Retorno --
# x: x tal que Lx = b

function resolve_triangular_inferior(L, b)
    n, = size(A) # dimensão da matriz L
    x = zeros(n, 1) # criamos uma matriz coluna n x 1
    
    # resolução do sistema por substituição direta
    for i = 1:n
        x[i] = b[i]
        for j = 1:i - 1
            x[i] -= L[i, j] * x[j]
        end
        x[i] /= L[i, i]
    end
    
    return x
end

resolve_triangular_inferior (generic function with 1 method)

##### iv. Decomposição LU

In [5]:
# Função para decompor a matriz densa A em uma multiplicação de matrizes triangulares n x n

# -- Parâmetros --
# A: matriz densa n X n

# -- Retorno --
# L, U: matrizes triangulares L e U tal que A = L * U

function decomposicao_lu(A)
    n, = size(A) # dimensão da matriz A
    L = Matrix{Float64}(I, n, n) # L inicialmente eh uma matriz identidade n x n
    U = copy(A) # inicialmente criamos a matriz U como uma cópia da matriz densa A

    # percorremos a matriz U e L para preenche-las
    for i = 1:n 
        for j = i+1:n
            k = U[j, i] / U[i, i]
            L[j, i] = k
            U[j, :] -= k * U[i, :]
        end
    end
    
    return L, U  
end

decomposicao_lu (generic function with 1 method)

##### v. Inversa

In [6]:
# Função para achar a inversa de uma matriz n x n usando LU

# -- Parâmetros --
# A: matriz n x n

# -- Retorno --
# inv: matriz inversa de A

function calcula_inversa(A)  
    n, = size(A) # dimensão da matriz A
    inv = zeros(n, n) # criamos uma matriz n X n para ser a inversa
    L, U = decomposicao_lu(A) # decompomos a matriz A na multiplicação de duas triangulares (uma inferior e outra superior)
    
    for i = 1:n
        # criamos uma vetor com a posição y[i] sendo 1 para que a junção de todos os y's em uma matriz corresponda à matriz identidade
        id = zeros(n)
        id[i] = 1
        
        # fazemos com que cada multiplicação seja igual a uma linha da identidade, pois sabemos que A * inv = I
        y = resolve_triangular_inferior(L, id) # resolvemos o sistema triangular inferior e obtemos y tal que Ly = b
        x = resolve_triangular_superior(U, y) # resolvemos o sistema triangular superior e obtemos x tal que Ux = y -> L(Ux) = b
        
        # formatamos o resultado na matriz inversa
        for j = 1:n
            inv[j, i] = x[j]
        end
    end
    
    return inv

end

calcula_inversa (generic function with 1 method)

#### -- Complexidade --

Sabemos que em um loop a complexidade é o tamanho do loop * a complexidade do que tiver dentro do loop. Por isso, o algoritmo escrito acima na função $\textit{calcula_inversa}$ tem complexidade cúbica $O(n^3)$, devido à etapa das resoluções dos sistemas triangulares superior e inferior de complexidade quadrática dentro do loop de tamanho $n$.

#### Exemplos de uso das funções da biblioteca

Vamos definir uma função para conferir o resultado das matrizes que nossas funções encontrarão:

In [7]:
# Função para conferir a matriz encontrada como solução de sistemas lineares

# -- Parâmetros --
# A: matriz original
# b: matriz coluna de resultados
# x: resultado do sistema que encontramos

# -- Retorno --
# Imprime uma mensagem indicando se a matriz calculada está correta ou não

function confere_matriz(A, b, x) 
    n, = size(x) # dimensão da matriz x
    
    x_check = A \ b # resolvemos o sistema com o Julia para conferir nossos resultados
    
    erro = 0.00001 # definimos um erro máximo 
    
    println("Matriz encontrada")
    
    for i = 1:n 
        println(x[i, 1]) # exibimos os valores de cada posição
        
        if x[i, 1] - x_check[i, 1] > erro
            print("O valor da posição ", i, ", está incorreto")
        end
        
    end
        
    print("O resultado do sistema está correto!")
end

confere_matriz (generic function with 1 method)

##### i. Diagonal

In [8]:
# -- Exemplo 1 --

A = [1 0 0; 0 2 0; 0 0 3]
b = [4; 5; 6]

x = resolve_diagonal(A, b)

confere_matriz(A, b, x)

Matriz encontrada
4.0
2.5
2.0
O resultado do sistema está correto!

In [9]:
# -- Exemplo 2 --

A = [34 0 0; 0 500 0; 0 0 346]
b = [983; 15; 96]

x = resolve_diagonal(A, b)

confere_matriz(A, b, x)

Matriz encontrada
28.91176470588235
0.03
0.2774566473988439
O resultado do sistema está correto!

In [10]:
# -- Exemplo 3 --

A = [-156 0 0 0; 0 98 0 0; 0 0 770 0; 0 0 0 578]
b = [9; 5; -7; 23]

x = resolve_diagonal(A, b)

confere_matriz(A, b, x)

Matriz encontrada
-0.057692307692307696
0.05102040816326531
-0.00909090909090909
0.039792387543252594
O resultado do sistema está correto!

##### ii. Triangular superior

In [11]:
# -- Exemplo 1 --

A = [1 2 3; 0 4 5; 0 0 6]
b = [7; 8; 9]

x = resolve_triangular_superior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
2.25
0.125
1.5
O resultado do sistema está correto!

In [12]:
# -- Exemplo 2 --

A = [13 -21 35 67; 0 46 -75 98; 0 0 68 90; 0 0 0 45]
b = [76; -18; -19; 67]

x = resolve_triangular_superior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
-7.4517372723894475
-7.231763285024155
-2.25
1.488888888888889
O resultado do sistema está correto!

In [13]:
# -- Exemplo 3 --

A = [47 86 69 -43; 0 4 -5 2.4; 0 0 7.8 -9; 0 0 0 4.5]
b = [7.6; -1; -9; 7]

x = resolve_triangular_superior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
1.3428623386070193
-0.382051282051282
0.6410256410256411
1.5555555555555556
O resultado do sistema está correto!

##### iii. Triangular inferior

In [14]:
# -- Exemplo 1 --

A = [3 0 0; 4 5 0; 7 10 2]
b = [3; 4; 5]

x = resolve_triangular_inferior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
1.0
0.0
-1.0
O resultado do sistema está correto!

In [15]:
# -- Exemplo 2 --

A = [6 0 0; 7.8 5.4 0; 74 150 12]
b = [5.6; 4.5; 5.1]

x = resolve_triangular_inferior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
0.9333333333333332
-0.5148148148148146
1.104629629629627
O resultado do sistema está correto!

In [16]:
# -- Exemplo 3 --

A = [-25 0 0 0; 48 -53 0 0; 57 10 -22 0; 43 65 -12 39]
b = [93; 34; -55; 64]

x = resolve_triangular_inferior(A, b)

confere_matriz(A, b, x)

Matriz encontrada
-3.72
-4.010566037735849
-8.961166380789024
9.66955886880415
O resultado do sistema está correto!

##### iv. Decomposição LU

In [17]:
# -- Exemplo 1 --

A = [1 2 3; 4 5 6; 7 8 9]

L, U = decomposicao_lu(A)

println("L = ", L)
println("U = ", U)

norm(L * U - A) < 0.0001 ? println("A decomposição está correta") : println("A decomposição está incorreta") 

L = [1.0 0.0 0.0; 4.0 1.0 0.0; 7.0 2.0 1.0]
U = [1 2 3; 0 -3 -6; 0 0 0]
A decomposição está correta


In [18]:
# -- Exemplo 2 --

A = Float64[16 -31 84; 43 45 -86; 577 -78 19]

L, U = decomposicao_lu(A)

println("L = ", L)
println("U = ", U)

norm(L * U - A) < 0.0001 ? println("A decomposição está correta") : println("A decomposição está incorreta") 

L = [1.0 0.0 0.0; 2.6875 1.0 0.0; 36.0625 8.104724792985873 1.0]
U = [16.0 -31.0 84.0; 0.0 128.3125 -311.75; 0.0 0.0 -483.6020457866539]
A decomposição está correta


In [19]:
# -- Exemplo 3 --

A = Float64[14.2 93 4.5; 4.6 5.7 8; 8 9 1.3]

L, U = decomposicao_lu(A)

println("L = ", L)
println("U = ", U)

norm(L * U - A) < 0.0001 ? println("A decomposição está correta") : println("A decomposição está incorreta") 

L = [1.0 0.0 0.0; 0.323943661971831 1.0 0.0; 0.5633802816901409 1.7765092544542467 1.0]
U = [14.2 93.0 4.5; 0.0 -24.426760563380284 6.54225352112676; 0.0 0.0 -12.857585192873204]
A decomposição está correta


##### v. Inversa

In [20]:
# -- Exemplo 1 --

A = [1 2 3; 0 4 5; 0 0 6]

A_inv = calcula_inversa(A)

println("inv = ", A_inv)

norm(A * A_inv - I) < 0.0001 ? println("A inversa encontrada está correta") : println("A inversa encontrada está incorreta") 

inv = [1.0 -0.5 -0.08333333333333337; 0.0 0.25 -0.20833333333333331; 0.0 0.0 0.16666666666666666]
A inversa encontrada está correta


In [21]:
# -- Exemplo 2 --

A = Float64[1.2 76 8.5; 3.4 4.6 7.8; 8 9 1.3]

A_inv = calcula_inversa(A)

println("inv = ", A_inv)

norm(A * A_inv - I) < 0.0001 ? println("A inversa encontrada está correta") : println("A inversa encontrada está incorreta") 

inv = [-0.015016194668993807 -0.005214281238220727 0.12946849872659288; 0.013557131219374862 -0.015535284550108067 0.004568926250889699; -0.001449710478787885 0.13963985450518565 -0.05912948159288572]
A inversa encontrada está correta


In [22]:
# -- Exemplo 3 --

A = [6 -1 2 5; 7.8 5.4 0 4; -74 150 12 56; 57 -8 5 4]

A_inv = calcula_inversa(A)

println("inv = ", A_inv)

norm(A * A_inv - I) < 0.0001 ? println("A inversa encontrada está correta") : println("A inversa encontrada está incorreta") 

inv = [-0.03312649995647795 0.036776134992974274 -0.0007429836232731069 0.015033760678446634; -0.09365945858565768 -0.004880687399743901 0.00728372648254766 0.019982839876148687; 0.07495741056218032 -0.5749574105621807 0.026831345826235108 0.10562180579216361; 0.19103694400576987 0.1848754647533544 -0.008384212686056783 -0.05629266715577167]
A inversa encontrada está correta


### Exercício 2 | Problema de Valor de Contorno

_________________________________________________________________________________________________________________________________________________________________________________________

#### a. Sistema linear que aproxima pelo método de diferenças finitas

Dado o PVC $$y''(x) = 4x \\ y(0) = 5 \\ y(10) = 20$$ queremos montar o sistema linear que o aproxima pelo método de diferenças finitas com $n = 6$ intervalos na discretização. 

Conforme visto em aula, pelo método de diferenças finitas, sabemos que a expressão para a segunda derivada é dada por:

$$y''(x_k) = \frac{y(x_{k + 1}) - 2y(x_k) + y(x_{k - 1})}{2h^2}$$

Substituindo nossa expressão da segunda derivada no PVC, temos:

$$
y''(x_k) = 4(x_k)\\
4(x_k) = \frac{y(x_{k + 1}) - 2y(x_k) + y(x_{k - 1})}{2h^2}
$$

Além disso, também sabemos que $h = \frac{b - a}{n}$, onde $a \leq x \leq b$ e $n$ é o número de intervalos. 

No caso do PVC apresentado, como $a = 0$ e $b = 10$ temos $h = \frac{10 - 0}{6} = \frac{10}{6} = \frac{5}{3}$.

Substituindo o valor de $h$ na expressão e passando-o para o outro lado teremos:

$$
4(x_k) * 2 * h^2 = y(x_{k + 1}) - 2y(x_k) + y(x_{k - 1}) \\
4(x_k) * 2 * \frac{25}{9} = y(x_{k + 1}) - 2y(x_k) + y(x_{k - 1}) \\
(x_k) * \frac{200}{9} = y(x_{k + 1}) - 2y(x_k) + y(x_{k - 1})
$$


Visto que precisamos ter $6$ intervalos, vamos usar $m = 7$ pontos. Pelas condições iniciais de contorno já sabemos os pontos $(0, 5)$ e $(10, 20)$, ou seja, já sabemos que $x_0 = 0$ e $x_6 = 20$, bem como $y(x_0) = 5$ e $y(x_6) = 20$. Para obtermos os $5$ pontos faltantes, vamos considerar $k \in [1, 5]$, de forma que teremos os pontos $(x_1, y(x_1)), (x_2, y(x_2)), (x_3, y(x_3)), (x_4, y(x_4))$ e $(x_5, y(x_5))$. Com esses pontos e a equação anterior, conseguimos montar o seguinte sistema: 

$$
\begin{cases}
    (x_1) * \frac{200}{9} = y(x_{2}) - 2y(x_1) + y(x_{0}) \\
    (x_2) * \frac{200}{9} = y(x_{3}) - 2y(x_2) + y(x_{1}) \\
    (x_3) * \frac{200}{9} = y(x_{4}) - 2y(x_3) + y(x_{2}) \\
    (x_4) * \frac{200}{9} = y(x_{5}) - 2y(x_4) + y(x_{3}) \\
    (x_5) * \frac{200}{9} = y(x_{6}) - 2y(x_5) + y(x_{4})
\end{cases}
$$

onde sabemos que $y(x_0) = 5$, $y(x_6) = 20$ e $x_k = x_{inicial} + h * (k - 1)$.

#### b. Resolvendo o sistema

Queremos agora resolver o sistema que obtivemos na letra a. Para montar a matriz dos coeficientes, vamos utilizar a função construída em aula para a criação da matriz:

In [23]:
# Função para criar uma matriz quadrada n x n no modelo usado no método das diferenças finitas

# -- Parâmetros --
# n: dimensão da matriz quadrada

# -- Retorno --
# A: a matriz criada

function criacao_da_matriz(n)
    A = zeros(n, n)
    
    # "manual" nos extremos
    A[1, 1] = -2
    A[1, 2] = 1
    A[n, n - 1] = 1
    A[n, n] = -2
    
    # tridiagonal
    for i = 2:(n - 1)
        A[i, i] = -2
        A[i, i + 1] = 1
        A[i, i - 1] = 1
    end
    
    return A
end

criacao_da_matriz (generic function with 1 method)

Desejamos utilizar a função de diferenças finitas construída em aula para resolver o sistema, então vamos defini-la para a utilizarmos:

In [24]:
# Função que usa o método das diferenças finitas para resolver um sistema

# -- Parâmetros --
# n: dimensão da matriz quadrada

# -- Retorno --
# y_meio: os valores de y sem contar os extremos

function diferencas_finitas(n, sd, y_i, y_f, x_i, x_f)
    h = (x_f - x_i)/ (n - 1) # tamanho do intervalo
    
    A = criacao_da_matriz(n - 2) # criamos a matriz de coeficientes
    b = zeros(n - 2) # criamos a matriz de resultados
    
    for i = 1:(n - 2) # no meio
        b[i] = sd(x_i + i * h) * 2 * h^2
    end
        
    b[1] -= y_i
    b[n-2] -= y_f
    
    y_meio = A \ b # resolvemos o sistema
    
    return y_meio  # valores da função no "meio"
end

diferencas_finitas (generic function with 1 method)

Criando variáveis para as informações que temos, vamos ter:

In [25]:
n = 7 # numero de pontos
sd(x) = 4x # segunda derivada
y_i = 5.0 # y inicial
y_f = 20.0 # y final
x_i = 0.0 # x inicial
x_f = 10.0 # x final
y_meio = diferencas_finitas(n, sd, y_i, y_f, x_i, x_f) # calculamos os valores de y "no meio" usando o método das diferenças finitas

# Preenchemos os valores de x
h = (x_f - x_i) / (n - 1)
x = zeros(n,1)
for i = 1:n
    x[i] = x_i + h * (i - 1)
end

# Preenchemos os valores de y com os pontos extremos que temos e o y do meio calculado
y = [y_i; y_meio; y_f]

7-element Vector{Float64}:
    5.0
 -208.5493827160494
 -385.0617283950617
 -487.50000000000006
 -478.82716049382725
 -322.0061728395063
   20.0

Então vamos ter como solução do sistema:

In [26]:
# Exibimos os pontos encontrados
for i = 1:length(y)
    println("y(", x[i], ") = ", y[i])
end

y(0.0) = 5.0
y(1.6666666666666667) = -208.5493827160494
y(3.3333333333333335) = -385.0617283950617
y(5.0) = -487.50000000000006
y(6.666666666666667) = -478.82716049382725
y(8.333333333333334) = -322.0061728395063
y(10.0) = 20.0


#### c. Interpolação polinomial

Queremos usar interpolação polinomial com grau 2 ou grau 3 para descobrir $y(3.2345)$. Com a questão anterior, já conseguimos definir o conjunto de pontos $x$ e $y$.

Agora vamos definir a função responsável pela criação da matriz de Vandermonde e também a função para realizar a interpolação, possibilitando que consigamos obter os coeficientes do polinômio interpolador que desejamos:

In [27]:
# Função auxiliar para construir a matriz de Vandermonde

# -- Parâmetros --
# x: conjunto dos pontos do eixo x
# grau: grau que desejamos para ser uma das dimensões da matriz

function vandermonde(x, grau)
    n, = size(x) # quantidade de pontos
    V = zeros(n, grau + 1) # criamos uma matriz n X (grau + 1)
    
    # preenchemos a matriz criada
    for i = 1:n
        for j = 1:(grau + 1)
            V[i, j] = x[i] ^ (j - 1)
        end
    end
    
    return V
end

vandermonde (generic function with 1 method)

In [28]:
# Função para gerar os coeficientes do polinômio interpolador de uma função

# -- Parâmetros --
# x: conjunto de valores de x
# y: conjunto de valores de y
# grau: o grau do polinômio interpolador que desejamos

function interpolacao(x, y, grau)
    V = vandermonde(x, grau) # construímos a matriz de Vandermonde
    c = V \ y # resolve o sistema linear Vc = y
    return c # vetor de coeficientes
end

interpolacao (generic function with 1 method)

Podemos realizar a interpolação de grau $2$ e $3$ e obter os coeficientes do polinômio interpolador, o que possibilitará a construção de uma função aproximativa. Com isso, podemos calcular $f(3.2345)$ como queríamos:

In [29]:
c = interpolacao(x, y, 2) # obtemos os coeficientes do polinômio interpolador de grau 2
f = Polynomial(c) # montamos o polinômio interpolador
approx = f(3.2345) # avaliamos o valor no ponto desejado
println("Grau 2: ", approx)

c = interpolacao(x, y, 3) # obtemos os coeficientes do polinômio interpolador de grau 3
f = Polynomial(c) # montamos o polinômio interpolador
approx = f(3.2345) # avaliamos o valor no ponto desejado
println("Grau 3: ", approx)

Grau 2: -414.7306672222221
Grau 3: -376.2958400485001


Utilizando o polinômio de grau $2$, obtemos que $y(3.2345) \approx -414.7306672222221$. Já utilizando grau $3$, obtemos que $y(3.2345) \approx -376.29584004850015$.

### Exercício 3 | Problema da Temperatura de um lago

_________________________________________________________________________________________________________________________________________________________________________________________

#### a. Sistema linear

Queremos descobrir a temperatura em diferentes lugares no interior de um lago (vértices $x_1, x_2, x_3$ e $x_4$). Para isso, primeiramente vamos modelar o problema das temperaturas como um sistema linear $Ax = b$.

Como sabemos as temperaturas nas margens e também que quando o calor está em equilíbrio, a temperatura em cada vértice no interior do lago é a média das temperaturas dos 4 vértices vizinhos, podemos modelar o sistema da seguinte forma:

$$
\begin{cases}
    x_1 = (15 + 5 + x_2 + x_3) / 4 \\
    x_2 = (15 + 35 + x_1 + x_4) / 4 \\
    x_3 = (5 + 10 + x_1 + x_4) / 4 \\
    x_4 = (10 + 35 + x_2 + x_3) / 4 
\end{cases}
$$

$$
\begin{cases}
    x_1 - x_2/4 - x_3/4 = 20 / 4 \\
    x_2 - x_1/4 - x_4/4 = 50 / 4 \\
    x_3 - x_1/4 - x_4/4 = 15 / 4 \\
    x_4 - x_2/4 - x_3/4 = 45 / 4 
\end{cases}
$$

E passando esse sistema para a forma matricial temos:

$$
\begin{bmatrix}
        1 & -\frac{1}{4} & -\frac{1}{4} & 0 \\
        -\frac{1}{4} & 1 & 0 & -\frac{1}{4} \\
        -\frac{1}{4} & 0 & 1 & -\frac{1}{4} \\
        0 & -\frac{1}{4} & -\frac{1}{4} & 1 \\
\end{bmatrix}  
\begin{bmatrix}
        x_1 \\
        x_2 \\
        x_3 \\
        x_4 
\end{bmatrix} =
\begin{bmatrix}
        20/4 \\
        50/4 \\
        15/4 \\
        45/4
\end{bmatrix}
$$

#### b. Temperatura dos 4 vértices no interior

Queremos determinar a temperatura dos 4 vértices no interior usando LU. Para isso, vamos primeiramente definir as matrizes acima:

In [30]:
# matriz com os coeficientes
A = [
    1 -1/4 -1/4 0
    -1/4 1 0 -1/4
    -1/4 0 1 -1/4
    0 -1/4 -1/4 1
]

# matriz coluna de resultados das equações
b = [
    20/4
    50/4
    15/4
    45/4
]

4-element Vector{Float64}:
  5.0
 12.5
  3.75
 11.25

Agora precisamos utilizar nossa função de decomposição LU para resolver o sistema denso que temos. Seguindo a estrutura vista em aula, vamos criar a função $\textit{resolve_sistema_denso}$ que dada uma matriz A de entrada, realiza a decomposição LU dela e em seguida resolve os sistemas triangulares para chegar na solução final:

In [31]:
# Função para resolver um sistema denso

# -- Parâmetros --
# A: matriz densa n x n
# b: matriz coluna de resultados

# -- Retorno --
# x: x tal que Ax = b

function resolve_sistema_denso(A, b)
    L, U = decomposicao_lu(A) # decompomos a matriz A na multiplicação de duas triangulares (uma inferior e outra superior)   
    y = resolve_triangular_inferior(L, b) # resolvemos o sistema triangular inferior
    x = resolve_triangular_superior(U, y) # resolvemos o sistema triangular superior
    return x
end

resolve_sistema_denso (generic function with 1 method)

Vamos também definir uma função auxiliar para imprimir as temperaturas quando as encontrarmos:

In [32]:
# Função para imprimir as temperaturas de cada vértice

# -- Parâmetros --
# x: matriz de temperaturas dos vértices

# -- Retorno --
# Imprime os valores das temperaturas de cada vértice

function imprime_temperaturas(x)
    n, = size(x) # dimensão da matriz x
    println("-- Temperatura em cada vértice --")
    
    # imprimindo cada temperatura
    for i = 1:n
        println("x_", i, " = ", x[i])
    end
end

imprime_temperaturas (generic function with 1 method)

In [33]:
# Usando nossa função com a matriz A e b que obtivemos a partir do sistema
x = resolve_sistema_denso(A, b)

# Exibindo as temperaturas calculadas
imprime_temperaturas(x)

-- Temperatura em cada vértice --
x_1 = 13.125
x_2 = 20.625
x_3 = 11.875
x_4 = 19.375


#### c. Temperatura dos vértices no interior

Agora a temperatura das margens mudou e queremos discretizar o lago ainda mais e usar decomposição LU para descobrir a temperatura dos vértices. Como são muitas variáveis para os vértices, vamos criar uma função para criar o sistema linear do lago:

In [34]:
# Função para criar um sistema linear do lago

# -- Parâmetros --
# n: tamanho do lago (numero de nos)
# T: matriz de temperaturas na seguinte ordem [tmp_esq, tmp_dir, tmp_cima, tmp_baixo]

# -- Retorno --
# A: matriz de coeficientes
# b: matriz de resultados 

function cria_sistema_lago(n, T)
    A = zeros(n, n) # matriz de coeficientes
    b = zeros(n) # matriz de resultados das equações
    
    r = Int(sqrt(n))

    for i = 1:n
        sum = 0 # variável auxiliar para armazenarmos o valor de b[i]
        A[i, i] = 1 # a diagonal principal será toda preenchida com 1's porque o próprio x_i tem valor 1 na iésima equação
        margin = false # variável auxiliar para indicar se o vértice está nas margens
        
        # Caso vértice esteja na esquerda
        if (i % r) == 1
            if i + r <= n
                A[i, i + r] = -1/4
            end
            
            if i + 1 <= n
                A[i, i + 1] = -1/4
            end
            
            if i - r > 0
                A[i, i - r] = -1/4
            end
            
            sum += T[1]
            margin = true
            # println("Entrou na esquerda: ", i)
        end
        
        # Caso vértice esteja na direita
        if (i % r) == 0
            if i - 1 > 0
                A[i, i - 1] = -1/4
            end
            
            if i + r <= n
                A[i, i + r] = -1/4
            end

            if i - r > 0
                A[i, i - r] = -1/4
            end
            
            sum += T[2]
            margin = true
            # println("Entrou na direita: ", i)
        end
        
        # Caso vértice esteja em cima
        if i <= r
            if i + r <= n
                A[i, i + r] = -1/4
            end
            
            if i + 1 <= n && !margin
                A[i, i + 1] = -1/4
            end
            
            if i - 1 > 0
                A[i, i - 1] = -1/4
            end
            
            sum += T[3]
            margin = true
            # println("Entrou em cima: ", i)
        end
        
        
        # Caso vértice esteja em baixo
        if i > r^2 - r
            if i - r > 0
                A[i, i - r] = -1/4
            end
            
            if i + 1 <= n
                A[i, i + 1] = -1/4
            end
            
            if i - 1 > 0 && !margin
                A[i, i - 1] = -1/4
            end
            
            sum += T[4]
            margin = true
            # println("Entrou em baixo: ", i)
        end

        
        # Caso geral, se não for um nó da margem
        if !margin
            A[i, i - 1] = -1/4
            A[i, i + 1] = -1/4
            A[i, i + r] = -1/4
            A[i, i - r] = -1/4
            # println("Entrou no caso geral: ", i)
        end

        b[i] = sum / 4
    end

    return A, b
end

cria_sistema_lago (generic function with 1 method)

Podemos fazer um teste de corretude da nossa função com o sistema que montamos na letra a:

In [35]:
n = 4 # tamanho do lago (numero de nos)
tmp_cima = 15 # temperatura da margem superior
tmp_baixo = 10 # temperatura da margem inferior
tmp_esq = 5 # temperatura da margem esquerda
tmp_dir = 35 # temperatura da margem direita

A, b = cria_sistema_lago(n, [tmp_esq, tmp_dir, tmp_cima, tmp_baixo])

x = resolve_sistema_denso(A, b)

4×1 Matrix{Float64}:
 13.125
 20.625
 11.875
 19.375

Obtemos o mesmo resultado que obtivemos na questão anterior. Como nosso teste deu certo, vamos agora usar nossa função para criar o sistema do lago com as novas condições. Vamos obter a matriz de coeficientes A e a matriz de resultados b, tal que $Ax = b$. E em seguida, vamos usar nossa função $\textit{resolve_sistema_denso}$ para utilizarmos LU para descobrir a temperatura dos vértices:

In [36]:
n = 25 # tamanho do lago
tmp_cima = 20 # temperatura da margem superior
tmp_baixo = 30 # temperatura da margem inferior
tmp_esq = 25 # temperatura da margem esquerda
tmp_dir = 20 # temperatura da margem direita

A, b = cria_sistema_lago(n, [tmp_esq, tmp_dir, tmp_cima, tmp_baixo])

display(A) # imprime a matriz de coeficientes
display(b) # imprime a matriz de resultados

x = resolve_sistema_denso(A, b) # resolve o sistema com o A e b que obtivemos

25×25 Matrix{Float64}:
  1.0   -0.25   0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
 -0.25   1.0   -0.25   0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0   -0.25   1.0   -0.25   0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.25   1.0   -0.25      0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   -0.25   1.0       0.0    0.0    0.0    0.0    0.0
 -0.25   0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
  0.0   -0.25   0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0   -0.25   0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0   -0.25   0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0   -0.25      0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0   …   0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0.0    0.0    0.0    0.0    0.0
  0.0    0.0    0.0    0.0    0.0       0

25-element Vector{Float64}:
 11.25
  5.0
  5.0
  5.0
 10.0
  6.25
  0.0
  0.0
  0.0
  5.0
  6.25
  0.0
  0.0
  0.0
  5.0
  6.25
  0.0
  0.0
  0.0
  5.0
 13.75
  7.5
  7.5
  7.5
 12.5

25×1 Matrix{Float64}:
 22.656565656565654
 21.762286324786324
 21.28651903651904
 20.89359945609946
 20.46969696969697
 23.863976301476303
 23.106060606060602
 22.49019036519036
 21.818181818181817
 20.98518842268842
 24.69327894327894
 24.30778943278943
 23.75
 22.903749028749036
 21.65287490287491
 25.601350038850036
 25.68181818181818
 25.29827117327117
 24.393939393939394
 22.722562160062164
 27.030303030303024
 27.519862082362078
 27.367327117327115
 26.65117521367521
 24.84343434343434

Com as novas condições, a temperatura em cada vértice vai ser:

In [37]:
# Exibindo as temperaturas calculadas
imprime_temperaturas(x)

-- Temperatura em cada vértice --
x_1 = 22.656565656565654
x_2 = 21.762286324786324
x_3 = 21.28651903651904
x_4 = 20.89359945609946
x_5 = 20.46969696969697
x_6 = 23.863976301476303
x_7 = 23.106060606060602
x_8 = 22.49019036519036
x_9 = 21.818181818181817
x_10 = 20.98518842268842
x_11 = 24.69327894327894
x_12 = 24.30778943278943
x_13 = 23.75
x_14 = 22.903749028749036
x_15 = 21.65287490287491
x_16 = 25.601350038850036
x_17 = 25.68181818181818
x_18 = 25.29827117327117
x_19 = 24.393939393939394
x_20 = 22.722562160062164
x_21 = 27.030303030303024
x_22 = 27.519862082362078
x_23 = 27.367327117327115
x_24 = 26.65117521367521
x_25 = 24.84343434343434


#### d. Discretizar com mais nós

Sim, é possível discretizar com mais nós usando a função auxiliar $\textit{cria_sistema_lago}$ que criamos acima. Para saber qual é o maior número de nós que conseguimos discretizar e rodar em menos de 2 minutos usando decomposição LU para resolver o problema, vamos usar a macro @time do julia:

In [38]:
n = 2304  # tamanho do lago (número de nós)
tmp_cima = 20 # temperatura da margem superior
tmp_baixo = 30 # temperatura da margem inferior
tmp_esq = 25 # temperatura da margem esquerda
tmp_dir = 20 # temperatura da margem direita

A, b = cria_sistema_lago(n, [tmp_esq, tmp_dir, tmp_cima, tmp_baixo])

@time resolve_sistema_denso(A, b) # cronometra o tempo para resolver o sistema com o A e b que obtivemos

108.857935 seconds (40.38 M allocations: 183.049 GiB, 2.57% gc time)


2304×1 Matrix{Float64}:
 22.50227986378084
 21.516282386350642
 21.05373028287246
 20.80145913848
 20.6467557363487
 20.54335130181544
 20.469765641260608
 20.414908802696555
 20.372533414285353
 20.33886464945687
 20.31148965197738
 20.28879408835463
 20.269656502424517
  ⋮
 29.441460479441012
 29.395696656146523
 29.34025982419464
 29.27194884097753
 29.185965674976785
 29.074788563756076
 28.92595089582622
 28.717301153517948
 28.405907585820337
 27.89926151076327
 26.97196556205829
 24.99772013621916

Após alguns testes na minha máquina, descobrimos que o número máximo de nós que conseguimos discretizar e utilizar decomposição LU em menos de 2 minutos é $n = 2304$, levando $108.857935$ segundos aproximadamente. Um $n$ maior já exige mais de $2$ minutos de processamento.

### Exercício 4 | Problema de distribuição de água

_________________________________________________________________________________________________________________________________________________________________________________________

#### a. Modelagem sem o cano $x_9$ e resolução do sistema linear

Queremos determinar o fluxo de água em cada cano (as arestas da figura). Para isso, primeiramente vamos realizar a modelagem sem o cano pontilhado $x_9$.

Para criar a modelagem do nosso sistema, vamos considerar que o fluxo de entrada em cada nó é igual ao fluxo de saída, ou seja, a soma dos canos que entram deve ser igual ao que sai. Dessa forma, teremos o seguinte sistema:

$$
\begin{cases}
    x_1 = 7000 \\
    x_2 = 3500 \\
    x_3 = 9000 \\
    x_4 = x_1 + 30000 \\
    x_5 = x_3 + 3000 \\
    x_6 = x_4 + x_2 \\
    x_7 = x_5 + 3000 \\
    x_8 = x_6 + 500 + x_7
\end{cases}
$$

$$
\begin{cases}
    x_1 = 7000 \\
    x_2 = 3500 \\
    x_3 = 9000 \\
    x_4 - x_1 = 30000 \\
    x_5 - x_3 = 3000 \\
    x_6 - x_4 - x_2 = 0 \\
    x_7 - x_5 = 3000 \\
    x_8 - x_6 - x_7 = 500
\end{cases}
$$

Passando esse sistema para a forma matricial, teremos:

$$
\begin{bmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
        -1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
        0 & 0 & -1 & 0 & 1 & 0 & 0 & 0 \\
        0 & -1 & 0 & -1 & 0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 0 & -1 & 0 & 1 & 0 \\
        0 & 0 & 0 & 0 & 0 & -1 & -1 & 1
\end{bmatrix}  
\begin{bmatrix}
        x_1 \\
        x_2 \\
        x_3 \\
        x_4 \\
        x_5 \\
        x_6 \\
        x_7 \\
        x_8
\end{bmatrix} =
\begin{bmatrix}
        7000 \\
        3500 \\
        9000 \\
        30000\\
        3000 \\
        0 \\
        3000 \\
        500
\end{bmatrix}
$$

In [39]:
# matriz com os coeficientes
A = [
         1 0 0 0 0 0 0 0
         0 1 0 0 0 0 0 0
         0 0 1 0 0 0 0 0
         -1 0 0 1 0 0 0 0
         0 0 -1 0 1 0 0 0
         0 -1 0 -1 0 1 0 0
         0 0 0 0 -1 0 1 0
         0 0 0 0 0 -1 -1 1
]

# matriz coluna de resultados das equações
b = [
        7000
        3500
        9000
        30000
        3000
        0
        3000
        500
]

8-element Vector{Int64}:
  7000
  3500
  9000
 30000
  3000
     0
  3000
   500

Assim como na questão anterior, agora temos um sistema denso e gostaríamos de usar decomposição LU para resolvê-lo. Para isso, vamos usar novamente nossa função $\textit{sistema_denso}$:

In [40]:
# Usando nossa função com a matriz A e b que obtivemos a partir do sistema
resolve_sistema_denso(A, b)

8×1 Matrix{Float64}:
  7000.0
  3500.0
  9000.0
 37000.0
 12000.0
 40500.0
 15000.0
 56000.0

Dessa forma, obtemos que o fluxo de água em cada cano, em litros por minuto, é de:

$$
\begin{cases}
    x_1 = 7000 \\
    x_2 = 3500 \\
    x_3 = 9000 \\
    x_4 = 37000 \\
    x_5 = 12000 \\
    x_6 = 40500 \\
    x_7 = 15000 \\
    x_8 = 56000
\end{cases}
$$

#### b. Modelagem com o cano $x_9$ e tentativa de resolução do sistema linear

Agora também queremos determinar o fluxo de água em cada cano (as arestas da figura), porém realizando uma modelagem considerando o cano o cano pontilhado $x_9$. Novamente, para criar a modelagem do nosso sistema, vamos considerar que o fluxo de entrada em cada nó é igual ao fluxo de saída, ou seja, a soma dos canos que entram deve ser igual ao que sai:

$$
\begin{cases}
    x_1 = 7000 \\
    x_2 = 3500 \\
    x_3 = 9000 \\
    x_4 = x_1 + 30000 + x_9 \\
    x_5 + x_9 = x_3 + 3000 \\
    x_6 = x_4 + x_2 \\
    x_7 = x_5 + 3000 \\
    x_8 = x_6 + 500 + x_7
\end{cases}
$$

$$
\begin{cases}
    x_1 = 7000 \\
    x_2 = 3500 \\
    x_3 = 9000 \\
    x_4 - x_1 - x_9 = 30000 \\
    x_5 + x_9 - x_3 = 3000 \\
    x_6 - x_4 - x_2 = 0 \\
    x_7 - x_5 = 3000 \\
    x_8 - x_6 - x_7 = 500
\end{cases}
$$

Passando esse sistema para a forma matricial, teremos:

$$
\begin{bmatrix}
        1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
        0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\
        -1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & -1 \\
        0 & 0 & -1 & 0 & 1 & 0 & 0 & 0 & 1 \\
        0 & -1 & 0 & -1 & 0 & 1 & 0 & 0 & 0\\
        0 & 0 & 0 & 0 & -1 & 0 & 1 & 0 & 0\\
        0 & 0 & 0 & 0 & 0 & -1 & -1 & 1 & 0
\end{bmatrix}  
\begin{bmatrix}
        x_1 \\
        x_2 \\
        x_3 \\
        x_4 \\
        x_5 \\
        x_6 \\
        x_7 \\
        x_8
\end{bmatrix} =
\begin{bmatrix}
        7000 \\
        3500 \\
        9000 \\
        30000\\
        3000 \\
        0 \\
        3000 \\
        500
\end{bmatrix}
$$

Nesse caso, percebemos que o sistema possui $8$ equações e $9$ incógnitas, o que o caracteriza como um sistema indeterminado. Dessa maneira, na teoria não conseguimos encontrar soluções exatas para o fluxo de água em cada um dos canos. Para que isso fosse possível, teríamos que ter mais uma equação. Porém, na prática, a implementação de decomposicao_lu, resolve_triangular_inferior e resolve_triangular_superior considera que a matriz de entrada é quadrada (em uma matriz $\textit{n x m}$ a implementação pega somente a dimensão $n$ e considera que a matriz é $\textit{n x n}$). Por esse motivo, conseguiremos rodar nosso algoritmo de sistema denso para resolver esse problema:

In [41]:
# matriz com os coeficientes
A = [
         1 0 0 0 0 0 0 0 0
         0 1 0 0 0 0 0 0 0
         0 0 1 0 0 0 0 0 0
         -1 0 0 1 0 0 0 0 -1
         0 0 -1 0 1 0 0 0 1
         0 -1 0 -1 0 1 0 0 0
         0 0 0 0 -1 0 1 0 0
         0 0 0 0 0 -1 -1 1 0
]

# matriz coluna de resultados das equações
b = [
        7000
        3500
        9000
        30000
        3000
        0
        3000
        500
]

8-element Vector{Int64}:
  7000
  3500
  9000
 30000
  3000
     0
  3000
   500

In [42]:
# Usando nossa função com a matriz A e b que obtivemos a partir do sistema
resolve_sistema_denso(A, b)

8×1 Matrix{Float64}:
  7000.0
  3500.0
  9000.0
 37000.0
 12000.0
 40500.0
 15000.0
 56000.0

Como pudemos perceber, o resultado é o mesmo que se não houvesse a presença do cano $x_9$.